In [47]:
import csv
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zawar_keys import tapi_key,oapi_key

In [26]:
url='https://api.themoviedb.org/3/movie/top_rated?api_key={}'.format(tapi_key)
url_params={'page': 1}

In [27]:
response=requests.get(url,params=url_params)
response.status_code

200

In [28]:
def get_tmdb_toprated(params,api_key):
    url='https://api.themoviedb.org/3/movie/top_rated?api_key={}'.format(api_key)
    response=requests.get(url,params=params)
    return response.json()

In [29]:
response=get_tmdb_toprated(url_params,tapi_key)

In [57]:
response['total_pages']

357

In [38]:
#for i in response['results']:
response['results'][0]

{'popularity': 29.678,
 'vote_count': 2238,
 'video': False,
 'poster_path': '/2CAL2433ZeIihfX1Hb2139CX0pW.jpg',
 'id': 19404,
 'adult': False,
 'backdrop_path': '/yFryZAcSO1XEwENkYUquQ6EBGsD.jpg',
 'original_language': 'hi',
 'original_title': 'दिलवाले दुल्हनिया ले जायेंगे',
 'genre_ids': [35, 18, 10749],
 'title': 'Dilwale Dulhania Le Jayenge',
 'vote_average': 8.8,
 'overview': 'Raj is a rich, carefree, happy-go-lucky second generation NRI. Simran is the daughter of Chaudhary Baldev Singh, who in spite of being an NRI is very strict about adherence to Indian values. Simran has left for India to be married to her childhood fiancé. Raj leaves for India with a mission at his hands, to claim his lady love under the noses of her whole family. Thus begins a saga.',
 'release_date': '1995-10-20'}

In [44]:
def parse_toprated(toprated):
    parsed_data=[]
    for i in toprated['results']:
        data=[
            i['id'],i['title'],i['original_language'],i['vote_average'],i['vote_count'],i['popularity'],
            i['genre_ids'],i['release_date'],i['adult']
        ]
        parsed_data.append(data)
    return parsed_data

In [46]:
parsed_responses=parse_toprated(response)

In [66]:
with open("TMDB_toprated.csv", "w", newline="", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows([['id','title','original_language','avg_vote','vote_count','popularity','genres','release_date','adult']])

In [64]:
def save_data(file_name,data):
    with open(file_name, "a", newline="", encoding='utf-8') as a:
        writer = csv.writer(a)
        writer.writerows(data)

In [56]:
save_data('TMDB_toprated.csv',parsed_responses)

In [67]:
p=1
while p<=357:
    url_params['page']=p
    
    results=get_tmdb_toprated(url_params,tapi_key)
    
    parsed_results=parse_toprated(results)
    
    save_data("TMDB_toprated.csv",parsed_results)
    p+=1

In [68]:
print(p)

358
